# Adaptive PDE discretizations on Cartesian grids
## Volume : Algorithmic tools
## Part : Automatic differentiation
## Chapter : Known bugs and incompatibilities

The techniques of automatic differentiation technique play an essential role in the notebooks presented in this repository. 

[**Summary**](Summary.ipynb) of volume Algorithmic tools, this series of notebooks.

[**Main summary**](../Summary.ipynb) of the Adaptive Grid Discretizations 
	book of notebooks, including the other volumes.

# Table of contents
  * [1 Matrix multiplication and inversion](#1-Matrix-multiplication-and-inversion)
  * [2. In place modifications and aliasing](#2.-In-place-modifications-and-aliasing)
    * [2.1 Aliasing of the AD information](#2.1-Aliasing-of-the-AD-information)
    * [2.2 Non writeable AD information](#2.2-Non-writeable-AD-information)
  * [3. CPU/GPU generic programming](#3.-CPU/GPU-generic-programming)



**Acknowledgement.** Some of the experiments presented in these notebooks are part of 
ongoing research with Ludovic Métivier and Da Chen.

Copyright Jean-Marie Mirebeau, Centre Borelli, ENS Paris-Saclay, CNRS, University Paris-Saclay

## 0. Importing the required libraries

In [2]:
import sys; sys.path.insert(0,"..") # Allow importing agd from parent directory
#from Miscellaneous import TocTools; TocTools.displayTOC('ADBugs','Algo')

In [3]:
import numpy as np
import scipy.sparse.linalg
from matplotlib import pyplot as plt

In [4]:
import agd.AutomaticDifferentiation as ad

In [5]:
def reload_packages():
    from Miscellaneous.rreload import rreload
    global ad
    ad, = rreload([ad],rootdir='..',verbose=True)

## 1 Matrix multiplication and inversion

Please use the `ad.apply_linear_mapping` and `ad.apply_linear_inverse` functions in combination with `np.dot`, or scipy solve functions.

In [5]:
v = ad.Dense.denseAD( np.random.standard_normal((4,)),np.random.standard_normal((4,4)))
m0 = np.random.standard_normal((4,4))
m1 = scipy.sparse.coo_matrix( ([1.,2.,3.,4.,5.],([0,2,1,2,3],[0,1,2,2,3]))).tocsr()

In [6]:
# Fails
#print("np.dot looses AD:",np.dot(m0,v))
#print("scipy '*' looses AD:",m1*v) 

In [7]:
print("np.dot with AD:\n",ad.apply_linear_mapping(m0,v))
print("scipy '*' with AD:\n",ad.apply_linear_mapping(m1,v))

np.dot with AD:
 denseAD([-0.01976637  1.46311232  1.27839053 -2.12205759],
[[ 3.16664039 -0.75784294 -0.18709705  1.63509827]
 [ 2.16380293 -2.18928542 -1.99481354  1.25948805]
 [-0.16606859 -3.54782692 -0.18451567 -3.19084469]
 [-3.50113151  3.58096534 -0.7818147  -0.0984395 ]])
scipy '*' with AD:
 denseAD([ 1.10867957 -0.34451339  1.23976812 -5.52771281],
[[ 1.69270082 -1.43799726  0.84163382  0.29858197]
 [-1.73296337 -1.43141325  0.70120112 -3.76452452]
 [-4.77681911 -0.77621287  1.71193028 -4.24881762]
 [ 2.14755697  7.19516215  4.83927223  3.21523553]])


In [8]:
print("scipy solve with AD :\n",ad.apply_linear_inverse(scipy.sparse.linalg.spsolve,m1,v))

scipy solve with AD :
 denseAD([ 1.10867957 -0.623792    0.28318655 -0.22110851],
[[ 1.69270082 -1.43799726  0.84163382  0.29858197]
 [ 0.53323987 -0.61601492 -0.14213163 -0.88427022]
 [-0.41103355  0.18872302  0.12949924  0.12842473]
 [ 0.08590228  0.28780649  0.19357089  0.12860942]])


## 2. In place modifications and aliasing

The AD information often consists of very large arrays. In order to save time and memory, this information is not systematically copied and/or stored fully. It can take the form of a broadcasted array, or of an alias to another array. In that case a copy is necessary to enable modifications.

### 2.1 Aliasing of the AD information

When an operation leaves the AD information untouched, an alias is used. This can lead to bugs if in place modifications are used afterward.

In [9]:
x=ad.Dense.identity(constant=np.array([1.,2.]))
y=x+1 # Only affects the value, not the AD information

In [10]:
print("Values are distinct :", x.value is y.value)
print("AD information is shared :", y.coef is x.coef)

Values are distinct : False
AD information is shared : True


A modification of the aliased variable will impact the original one.

In [11]:
print(x[0])
y[0]*=2
print("Caution ! Shared AD information is affected :", x[0])

denseAD(1.0,[1. 0.])
Caution ! Shared AD information is affected : denseAD(1.0,[2. 0.])


Avoid this effect by making a copy.

In [12]:
x=ad.Dense.identity(constant=np.array([1.,2.]))
y=(x+1).copy()
print("AD information is distinct :", y.coef is x.coef)

AD information is distinct : False


Note that a similar effect arises with the `-` binary operator, but not with `*`or `/`. That is because the latter modify the AD information, which therefore must be copied anyway.

In [13]:
x=ad.Dense.identity(constant=np.array([1.,2.]))
print("AD information is shared :", (x-1).coef is x.coef)
print("AD information is distinct :", (x*2).coef is x.coef)
print("AD information is distinct :", (x/2).coef is x.coef)

AD information is shared : True
AD information is distinct : False
AD information is distinct : False


### 2.2 Non writeable AD information

When creating an dense AD variable, the coefficients may be non writeable (e.g. broadcasted) arrays.

In [14]:
x=ad.Dense.identity(constant=np.array([[1.,2.],[3.,4.]]),shape_bound=(2,))

In [15]:
x.coef.flags.writeable

False

In [16]:
# x+=1 # Fails because non-writeable

Make a copy to solve the issue.

In [17]:
y=x.copy()

In [18]:
y.coef.flags.writeable

True

In [19]:
y+=1

## 3. CPU/GPU generic programming

The agd library allows CPU/GPU generic programming to some extent. Here are the guidelines to make this approach work.

*Make a copy of the numpy array module*

In [20]:
xp = np 

*Activate GPU acceleration.*
If uncommented, the following line will replace the module xp with np, and modify its other arguments in a custom manner intended for easy interaction.

In [ ]:
#xp,plt = map(ad.cupy_friendly,[xp,plt])

*Create basic arrays using `xp`.* Basic arrays are arrays of zeros, of ones, arange, linspace, etc

In [30]:
x = xp.linspace(0,2*np.pi)

*Use numpy's overloading mechanisms.* These mechanisms will dispatch the function calls to cupy, or to the AutomaticDifferentiation module of the agd library, depending on the data type (array from numpy, cupy, or ad).

In [31]:
y = np.sin(x)

*Use `ad.asarray` and `ad.array`.* Stacking arrays using np.array will not work for AD or cupy arrays.

In [32]:
xy = ad.array([x,y])

*Use functions that accept both numpy and cupy arrays.* Or modify them for that purpose, as we did for the member functions of the pyplot module using the `ad.cupy_friendly` function.

In [33]:
plt.plot(*xy);

*If needed, convert cupy array's to numpy arrays.* Using the `get` member function of the `cupy.ndarray` class, or using `ad.cupy_generic.cupy_get` (which leaves non-cupy variables unchanged).

In [36]:
type(ad.cupy_generic.cupy_get(y))

numpy.ndarray